In [1]:
from qutip import*
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
from scipy.optimize import curve_fit
from tcsim.system import*
from tcsim.visualization import*
from tcsim.gates import*

t1 = 263
gamma_a_loss = 1 / t1  # t1 = 100mu s
a_loss = np.sqrt(gamma_a_loss) * destroy(N)


def gauss_1(x, amp1, cen1, sigma1):
    return amp1 * (np.exp((-1.0 / 2.0) * (((x - cen1) / sigma1) ** 2)))

def gauss_3(x, amp1, cen1, sigma1, amp2, cen2, sigma2, amp3, cen3, sigma3):
    """ Fitting Function"""
    return amp1 * (np.exp((-1.0 / 2.0) * (((x - cen1) / sigma1) ** 2))) + \
           amp2 * (np.exp((-1.0 / 2.0) * (((x - cen2) / sigma2) ** 2))) + \
           amp3 * (np.exp((-1.0 / 2.0) * (((x - cen3) / sigma3) ** 2)))

# calculated 1D char func
def char_func_cut(state, xvec, axis = 1):
    """Calculate the Characteristic function as a 2Dgrid (xvec, xvec) for a given state.

    Args:
        state (Qobject): State of which we want to calc the charfunc
        xvec (_type_): array of displacements. The char func will be calculated for the grid (xvec, xvec)

    Returns:
        tuple(ndarray, ndarray): Re(char func), Im(char func)
    """
    cfReal = np.empty((len(xvec)))
    

    for i, alpha_x in enumerate(xvec):
            expect_value = expect(displace(N, 1j*alpha_x*axis),state)
            cfReal[i] =  np.real(expect_value)

    return cfReal

def transmission (t):
    return np.exp(-t/t1)

def relative_to_t1(t):
    return t/t1

def calc_squeez_parameter(dB):
    return np.log(10**(dB/10))/2

# find the min value of the char function, assuming the blobs are align on the X axis (works for odd cat.)
# def char_negativiy(state):
#     xvec = np.linspace(-6,6,100)
#     xcut = char_func_cut(state, xvec, 1)
#     return np.min(xcut) 
 
def lossy_channel(state, t_list):
    rhos_loss = mesolve(Ic,state, t_list, c_ops = a_loss).states
    return rhos_loss

def extract_amp_triple_gaussian_fit(state):
    xvec = np.linspace(-6,6,100)
    amp1 = 0.3
    sigma1 = 1
    cen1 = -2.5
    amp2 = 0.8
    sigma2 = 1
    cen2 = 0
    amp3 = 0.3
    sigma3 = 1
    cen3 = 2.5
    initial_guess = [amp1, cen1, sigma1, amp2, cen2, sigma2, amp3, cen3, sigma3]
    bounds = ([0, -np.inf, -np.inf,-np.inf, -np.inf, -0.9,0, -np.inf, -np.inf ],[1, np.inf, 1.2,np.inf, np.inf, 1.2,1, np.inf, 1.1 ]) 

    y = char_func_cut(state, xvec)
    #plt.plot(xvec,y)
    popt, pcov = curve_fit(gauss_3, xvec, y, p0=initial_guess, bounds=bounds, maxfev = 5000)
    
    print("blob position is:", popt[1])
    print("sigma is:", popt[2])
    return (popt[0]+popt[6])/2


def get_data(states, t_list):
    blob_amps=[]
    rhos_loss_stored = []
    for i, state in enumerate(states):
        blob_amps.append([])
        rhos_loss = lossy_channel(state,t_list)
        rhos_loss_stored.append(rhos_loss)
        for j, rho in enumerate(rhos_loss):
            blob_amp = extract_amp_triple_gaussian_fit(rho)
            blob_amps[i].append(blob_amp) # minus sign cause i want to take a look at the positive blobs to compare it 

            
    return blob_amps, rhos_loss_stored


def char_func_grid(state, xvec):
    """Calculate the Characteristic function as a 2Dgrid (xvec, xvec) for a given state.

    Args:
        state (Qobject): State of which we want to calc the charfunc
        xvec (_type_): array of displacements. The char func will be calculated for the grid (xvec, xvec)

    Returns:
        tuple(ndarray, ndarray): Re(char func), Im(char func)
    """
    cfReal = np.empty((len(xvec),len(xvec)))
    cfImag = np.empty((len(xvec),len(xvec)))
    N = state.dims[0][1]

    for i, alpha_x in enumerate(xvec):
        for j, alpha_p in enumerate(xvec):
            expect_value = qt.expect(qt.displace(N, alpha_x +1j*alpha_p),qt.ptrace(state,1))
            cfReal[i,j] =  np.real(expect_value)
            cfImag[i,j] =  np.imag(expect_value)

    return cfReal,cfImag  

def squeezingStateOneStep(uvs):
    u_1 = uvs[0]
    v_1 = uvs[1]

    psi1 = V_ideal_operator(v_1)*U_ideal_operator(u_1)*vac
 
    return psi1 

def squeezingStateTwoStep(uvs):
    u_1 = uvs[0]
    v_1 = uvs[1]
    u_2 = uvs[2]
    v_2 = uvs[3]
    
    
    psi1 = U_ideal_operator(u_1)*vac
    psi2 = V_ideal_operator(v_1)*psi1
    psi3 = U_ideal_operator(u_2)*psi2
    psi4 = V_ideal_operator(v_2)*psi3
 
    return psi4

def squeezingStateThreeStep(uvs):
    u_1 = uvs[0]
    v_1 = uvs[1]
    u_2 = uvs[2]
    v_2 = uvs[3]
    u_3 = uvs[4]
    v_3 = uvs[5]

    psi1 = V_ideal_operator(v_1)*U_ideal_operator(u_1)*vac
    psi2 = V_ideal_operator(v_2)*U_ideal_operator(u_2)*psi1
    psi3 = V_ideal_operator(v_3)*U_ideal_operator(u_3)*psi2
 
    return psi3

In [2]:
### Initialize vacuum and squeezed vacuum states via hastrup
rho_0 = ptrace(vac,1)
rho_1 = ptrace(squeezingStateOneStep([1,-0.606]),1)
rho_2 = ptrace(squeezingStateTwoStep([ 1.93872301,  0.4409296,  -0.57281612, -1.06157337] ),1)
rho_3 = ptrace(squeezingStateThreeStep([-0.84571552,  0.61335235,  2.63648988,  0.30560449, -0.91694799, -0.80353261]),1)
states = [rho_0, rho_1, rho_2, rho_3]


### turn them into the according cat states
### in experiment, we displaced the states as if they were ideal squeezed states. Its only fair to do the same here if we want to compare the two.
alphas =[1.8, 1.4297908225037066, 1.0477857919875688, 0.7948268052359626]
cats = []
for i, state in enumerate(states):
    cat = ((displace(N,1j*alphas[i]) + displace(N, -1j*alphas[i]))*state*(displace(N,1j*alphas[i]) + displace(N, -1j*alphas[i])).dag()).unit()
    cats.append(cat)


# check that the states are indeed correct.
plot_char(tensor(fock(2,0)*fock(2,0).dag(),cats[0]), max_alpha=6, npts=300)



In [ ]:
t1 = 263
gamma_a_loss = 1 / t1  # t1 = 100mu s
a_loss = np.sqrt(gamma_a_loss) * destroy(N)

# time evolution span
ts = [     1,  10,  16,  30,  50,  70, 100,
       150, 200]
#ts = np.linspace(0,200, 15) # values are chosen such that we are between transmission of 0.5 (70) and 1 (0.1)

# get the wigner negativity of the state for each point in time
#blob_amps, rhos = get_data(cats[1], ts)

rhos_loss = lossy_channel(cats[3],ts)
for j, rho in enumerate(rhos_loss):
    print(ts[j])
    blob_amp = extract_amp_triple_gaussian_fit(rho)

# use to plot the negativity vs "transmission" exp(-t/T1)
transmis = []
for i in ts:
    transmis.append(transmission(i))

# calculate points in time as fraction of T1 
rel_t1 = []
for i in ts:
    rel_t1.append(relative_to_t1(i))

In [ ]:
import matplotlib as mpl
mpl.rcParams['legend.loc'] = 'best'
mpl.rcParams["axes.linewidth"] = 2.0
mpl.rcParams['legend.frameon'] = True

mpl.rcParams['font.sans-serif'] ='Arial'
mpl.rcParams['axes.labelsize']= 8
mpl.rcParams['legend.fontsize'] = 14

fig, ax = plt.subplots(figsize = (10,6))

ax.tick_params(direction = "in", bottom=True, top=True, left=True, right=True, length=4, width=2, labelsize = 14)
#ax.grid();
times = ts*1e3
#general_figure()plt.plot(transmis,negativity_loss[0], 'o', label = '0')
ax.plot(times,blob_amps[0], '--', color = '#ef476f', label = '0 dB')
ax.plot(times,blob_amps[1], '--', color = '#f78c6b',label = '2 dB')
ax.plot(times,blob_amps[2], '--', color = '#06d6a0',label = '4.8 dB')
ax.plot(times,blob_amps[3], '--', color = '#118ab2',label = '7.1 dB')
#ax.plot(rel_t1,negativity_loss[4], '>', color = '#073b4c',label = '12 dB')
# ax.plot(rel_t1,negativity_loss2[4], '--^', color = '#073b4c',label = '8 dB')
# ax.plot(rel_t1,negativity_loss2[5], '--v', color = '#118ab2',label = '10 dB')

ax.set_xlim()

ax.set_title(f"squeezed cat+ state evolving under photon loss, coh:1.8 ", fontsize=18)
#ax.set_title(f"cat- state with different degrees of squeezing different alpha, but same state overlap under loss", fontsize=18)
ax.set_xlabel(r"$t/T_1$", fontsize=16)
ax.set_ylabel("char blob amp", fontsize=16)

ax.legend()
